In [1]:
import pandas as pd
import subprocess
import shutil
import os

from pathlib import Path
from glob import glob

In [2]:
pd.options.display.max_columns = 50

In [3]:
df = pd.read_excel('/home/tee/temp/附件下载_REAPER API 填写表（收集结果）/0-REAPER API 填写表（收集结果）.xlsx')
# df.head(1)

In [4]:
# 清洗列
# 去掉空白列
# 重新定义df
# 全部cell作strip
real_list_count = 16
col_list = [li for li in df.columns.tolist() if 'Unnamed' not in li]
assert df.columns.tolist() != real_list_count

df = df[col_list]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.apply(lambda x: x.replace('：', ':') if x.dtype == "object" else x)


In [5]:
df.head(1)

,提交者（自动）,提交时间（自动）,你的昵称（必填）,API 所属分类（必填）,API 名称（必填）,API 描述（必填）,API 参数（必填）,API 返回值（必填）,C++ 代码范例,EEL 代码范例,Lua 代码范例,Python 代码范例,更详细的代码应用示范,API 使用效果图,API 使用注意事项,相关 API
0,noiZ,2020-10-22 18:39:34,noiZ,Mediaitem Management,GetSelectedMediaItem,通过序号获取已选中的一个item对象,proj: 工程对象，一般填0\nindex: 被选中item的序号，以0为起点,item: 获取到的item对象,NaN,NaN,"local item=reaper.GetSelectedMediaItem(0, 0)",NaN,NaN,NaN,Reaper中所有涉及到item次序的参数，均按照三个原则：\n1、处于不同轨道的item，...,NaN


In [6]:
# 清洗分类名字
# 空格用-代替
# 建立分类文件夹
cate = list(set(df['API 所属分类（必填）'].tolist()))
cate = [c.strip() for c in list(filter(None, cate))]
cate_folder = [c.replace(' ', '-') for c in cate]

for cf in cate_folder:
    Path(f'./source/{cf}').mkdir(exist_ok=True)

In [7]:
# 根据分类切割成multi df
cate_group = df.groupby(df['API 所属分类（必填）'])
# ### 取一个df检查用
# df_temp = cate_group.get_group(cate[0]).head(1)
# df_temp

In [8]:
# 加载rst模板
with open('./api-template.rst') as f:
    template = f.read()

# ### 取df_temp检查用
# for i, row in df_temp.iterrows():
#     print(row)
#     break

# 清空图片文件以便同步
[os.remove(x) for x in glob('./source/_static/*')]

# 检查是否为整数
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [9]:
for i, c in enumerate(cate):
    df_content = cate_group.get_group(cate[i])
    for i, row in df_content.iterrows():
        ### 基础信息
        author = row['你的昵称（必填）']
        date = row['提交时间（自动）'].strftime('%Y-%m-%d')
        fn_name = row['API 名称（必填）']
        fn_desc = row['API 描述（必填）']

        one_tab = '   '

        ### 参数&返回值
        ### 做成字典
        para_items = row['API 参数（必填）']
        para_items = para_items.split('\n')
        rtn_items = row['API 返回值（必填）']
        rtn_items = rtn_items.split('\n')

        if any(ext in ''.join(para_items) for ext in [':', '：']):
            # para
            para_dict = {}
            for item in para_items:
                item = item.replace(': ', ':')
                k, v = item.split(":", maxsplit=1)
                para_dict[k] = v
                
            para_name_list_text = ', '.join(list(para_dict.keys()))
            para_name_list_text = '*' + para_name_list_text + '*'

            para_text = ''
            count = 1
            for k, v in para_dict.items():
                para_text += f'**{k}**\n{one_tab*3 + one_tab*count}{v}\n{one_tab*3 + one_tab*(count-1)}'
                count += 1
            para_text = para_text[:para_text.rfind('\n')]
            
        elif any(ext in ''.join(rtn_items) for ext in [':', '：']):
            # return
            rtn_dict = {}
            for item in rtn_items:
                item = item.replace(': ', ':')
                k, v = item.split(":", maxsplit=1)
                rtn_dict[k] = v

            rtn_text = ''
            count = 1
            for k, v in rtn_dict.items():
                rtn_text += f'**{k}**\n{one_tab*3 + one_tab*count}{v}\n{one_tab*3 + one_tab*(count-1)}'
                count += 1
            rtn_text = rtn_text[:rtn_text.rfind('\n')]

        else:
            para_text = one_tab + para_items[0]
            rtn_text = one_tab + rtn_items[0]
            para_name_list_text = ''


        ### 代码范例
        code_block_text = ''
        code_block_text_ext = ''
        if pd.isnull(row['C++ 代码范例']) is False:
            code_block_text += f"{one_tab}.. code-block:: c++\n{one_tab*2}:caption: C++:\n\n{one_tab*3}{row['C++ 代码范例']}\n\n"
        if pd.isnull(row['EEL 代码范例']) is False:
            code_block_text += f"{one_tab}.. code-block:: javascript\n{one_tab*2}:caption: EEL:\n\n{one_tab*3}{row['EEL 代码范例']}\n\n"
        if pd.isnull(row['Lua 代码范例']) is False:
            code_block_text += f"{one_tab}.. code-block:: lua\n{one_tab*2}:caption: Lua:\n\n{one_tab*3}{row['Lua 代码范例']}\n\n"
        if pd.isnull(row['Python 代码范例']) is False:
            code_block_text += f"{one_tab}.. code-block:: python\n{one_tab*2}:caption: Python:\n\n{one_tab*3}{row['Python 代码范例']}\n\n"
            
        if pd.isnull(row['更详细的代码应用示范']) is False:
            code_block_text_ext += f"{one_tab}.. rubric:: 更多范例\n{one_tab}.. code-block::\n\n"
            for tl in row['更详细的代码应用示范'].split('\n'):
                code_block_text_ext += f'{one_tab*4}{tl}\n'
            
            code_block_text_ext += '\n\n'
            

        code_block_text = code_block_text[:code_block_text.rfind('\n')]
        code_block_text_ext = code_block_text_ext[:code_block_text_ext.rfind('\n')]

        ### 注意事项
        note_text = ''
        if pd.isnull(row['API 使用注意事项']) is False:
            note_text += f".. note::\n"
            for note_line in row['API 使用注意事项'].split('\n'):
                # 检查是否序号，转换成rst格式
                if RepresentsInt(note_line[:1]):
                    if any(ext in note_line[1:2] for ext in [',', '、', '.']):
                        note_line = '#. ' + note_line[2:].strip()

                note_text += f'{one_tab*3}{note_line}\n\n'

        ### 效果图
        image_text = ''
        img_name_list = []
        if pd.isnull(row['API 使用效果图']) is False:
            file_list = glob('/home/tee/temp/附件下载_REAPER API 填写表（收集结果）/*')
            for file in file_list:
                img_checkname = row['API 使用效果图'].replace(':', '_')
                img_checkname = img_checkname.replace('/', '_')
                if img_checkname in file:
                    split_point = file.split('/')[-1].split('-')[2]
                    img_name = file[file.index(split_point):]
                    img_name = img_name.replace(' ', '-')
                    
                    img_name_list.append(img_name)
                    shutil.copyfile(file, './source/_static/'+img_name)
                    
            image_text += f"{one_tab}.. rubric:: 使用效果\n"
            for inl in img_name_list:        
                image_text += f"{one_tab}.. image:: ../../_static/{inl}\n\n\n"
            
        
        ### 相关API

        ### 测试写入rst
        content = template
        content = content.replace('{author}', author)
        content = content.replace('{date}', date)
        content = content.replace('{fn_name}', fn_name)
        content = content.replace('{fn_desc}', fn_desc)
        content = content.replace('{para_name_list_text}', para_name_list_text)
        content = content.replace('{para_text}', para_text)
        content = content.replace('{rtn_text}', rtn_text)
        content = content.replace('{code_block_text}', code_block_text)
        content = content.replace('{code_block_text_ext}', code_block_text_ext)
        content = content.replace('{note_text}', note_text)
        content = content.replace('{image_text}', image_text)

        cate_folder_selected = row['API 所属分类（必填）'].replace(' ', '-')
        with open(f'./source/{cate_folder_selected}/{fn_name}.rst', 'w+') as f:
            f.write(content)

In [10]:
# 各分类目录创建index.rst
# for i, cf in enumerate(cate_folder):
#     index_exists = Path(f'./source/{cf}/index.rst').is_file()
#     if index_exists is False:
#         index_content = f'{cate[0]}\n{"="*30}\n\n.. toctree::\n   :glob:\n\n*'
#         with open(f'./source/{cf}/index.rst', 'w') as f:
#             f.write(index_content)
            

In [11]:
# make_command = 'make clean && make html'
# make_command = make_command.split(' ')

# subprocess.run(make_command, )

In [12]:
pd.DataFrame(
    {
        'cate': [c.replace(' ', '-')+'/*' for c in cate]
    }
).to_clipboard(index=False, header=False)